In [1]:
import geopandas as gpd
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',10)
pd.set_option('display.max_colwidth', None)

In [4]:
#重心座標カラムを追加する
def center(gdf, gdf_osyu_polygon):
    gdf = gdf.rename(columns={'メッシュ' : 'Meshcode', '土地利用種' : 'lu_code', '撮影年月日' : 'lu_year'})
    src_proj_gdf = 4612 # 変換前の座標系を指定
    src_proj_gdf_osyu_polygon = 6668 # 変換前の座標系を指定
    dst_proj = 4326  # 変換後の座標系を指定

    # ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
    gdf.crs = f'epsg:{src_proj_gdf}'  # 変換前座標を指定
    gdf_osyu_polygon.crs = f'epsg:{src_proj_gdf_osyu_polygon}' # 変換前座標を指定
    gdf = gdf.to_crs(epsg=dst_proj)  # 変換後座標に変換
    gdf_osyu_polygon = gdf_osyu_polygon.to_crs(epsg=dst_proj) # 変換後座標に変換



    #100mメッシュのデータを奥州市のポリゴンと接しているものだけ空間結合することで残す
    gdf_osyu_polygon = gdf_osyu_polygon[gdf_osyu_polygon['N03_004']=='奥州市']
    gdf = gpd.sjoin(gdf, gdf_osyu_polygon, how='inner', op='intersects')
    #sjoinにはignore_indexの引数はないのでrest_indexを行う
    gdf.reset_index(drop=True, inplace=True)
    #奥州市のポリゴンのカラムはいらないので削除
    gdf = gdf[['Meshcode', 'lu_code', 'lu_year', 'geometry']]

    center_list = []
    gdf.apply(lambda row: center_list.append(list(row['geometry'].centroid.coords)[0]), axis=1)

    center_df = DataFrame(center_list, columns=['longitude', 'latitude'])
    gdf = gdf.join(center_df)
    return gdf

In [16]:
#降水量、気温のカラムを追加する
def tempreture_rain(gdf, gdf_weather):
  #データフレームをジオデータフレームにする
  gdf_weather = gpd.GeoDataFrame(gdf_weather, geometry='geometry')

  #座標系を変更
  src_proj = 4612 # 変換前の座標系を指定
  dst_proj = 4326  # 変換後の座標系を指定
  # ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
  gdf_weather.crs = f'epsg:{src_proj}'  # 変換前座標を指定
  gdf_weather = gdf_weather.to_crs(epsg=dst_proj)  # 変換後座標に変換

  avg_rain = []
  max_rain = []
  max_temperature = []
  min_temperature = []
  for i, row in gdf_weather.iterrows():
      avg_rain.append(row.iloc[1:13].mean())
      max_rain.append(row.iloc[1:13].max())
      max_temperature.append(row.iloc[15:49:3].max())
      min_temperature.append(row.iloc[15:49:3].min())
  gdf_weather['avg_rainfall'] = avg_rain
  gdf_weather['max_rainfall'] = max_rain
  gdf_weather['max_temperature'] = max_temperature
  gdf_weather['min_temperature'] = min_temperature
  #100mメッシュに1kmメッシュを結合
  gdf_merge = gpd.sjoin(gdf, gdf_weather[[ 'G02_053', 'max_temperature', 'min_temperature', 'avg_rainfall', 'max_rainfall', 'geometry']], how='left', op="intersects" )
  #結合の時に１kmメッシュの間にある100mのポリゴンは２回カウントされるのでindexが重複するものは削除する
  gdf_merge = gdf_merge[~gdf_merge.index.duplicated()]
  #結合するdataframeのindexが残っていると後で結合できないのでindex_rightを削除
  gdf_merge = gdf_merge.drop(columns=['index_right'])
  return gdf_merge

In [6]:
#正しく空間結合出来ているか確認

#row_indexに確かめる行のindexを指定してください, latitudeにgdfで指定したデータフレームの緯度となるカラム名をしていてください。
def cheack_join(gdf, row_index, latitude):
    # 座標が存在するポイントを作成
    #緯度経度座標のカラムがどこにあるか探す
    columns = gdf.columns.to_list()
    column = columns.index(f'{latitude}')

    point = Point(gdf.iloc[row_index, column+1], gdf.iloc[row_index, column])# 座標を指定

    # ポリゴンデータの中でポイントが含まれる行を抽出
    contains_point = gdf_5841.contains(point)

    # ポイントが存在するポリゴンデータを抽出
    selected_polygons = gdf_5841[contains_point]

    # 選択されたポリゴンデータの情報を表示
    return selected_polygons

In [7]:
#道路舗装データの空間結合

def road_data(gdf, gdf_road):
  #道路劣化調査の奥州市のみのデータフレームを作る
  gdf_road_osyu = gdf_road[gdf_road['locate_area2'] == '奥州市']
  gdf_road_osyu = gpd.GeoDataFrame(gdf_road_osyu)

  #データフレームの緯度経度座標をstr型からflaot型に変換
  gdf_road_osyu['locate_self_long'] = gdf_road_osyu['locate_self_long'].replace({'':0}).astype(float)
  gdf_road_osyu['locate_self_lat'] = gdf_road_osyu['locate_self_lat'].replace({'':0}).astype(float)

  #データフレームの緯度経度座標が空の行を削除する
  gdf_road_osyu = gdf_road_osyu[gdf_road_osyu['locate_self_lat'] != 0]

  #緯度経度座標をpoint型にしたカラムを増やす
  gdf_road_osyu['geometry'] = gdf_road_osyu.apply(lambda x: Point(x['locate_self_long'], x['locate_self_lat']), axis=1)

  #座標系を設定
  if gdf_road_osyu.crs is None:
      # 座標系をEPSG:4326に設定
      gdf_road_osyu.set_crs(epsg=4326, inplace=True)

  #ポリゴンデータにポイントデータがあれば空間結合
  joined_gdf = gpd.sjoin(gdf_osyu, gdf_road_osyu, op='contains')
  #2つのデータフレームで同じカラム（geometry)がないと結合することが出来ない

  #indexが重複しているものを削除する
  joined_gdf_test = joined_gdf[~joined_gdf.index.duplicated(keep='first')]

  #indexが重複しているものがないか確認
  duplicate_rows = joined_gdf_test[joined_gdf_test.index.duplicated(keep=False)]

  #等しい100mメッシュ内に複数入るfacility_idを削除
  filtered_gdf_road_osyu = gdf_road_osyu[gdf_road_osyu['facility_id'].isin(joined_gdf_test['facility_id'])]

  #100mメッシュ内に入るfacility_idを一つになるように空間結合
  gdf_osyu = gpd.sjoin(gdf_osyu, filtered_gdf_road_osyu, op='contains', how='left')

  #結合した際にfiltered_gdf_road_osyuのindexを削除
  gdf_osyu = gdf_osyu.drop('index_right', axis=1)
  return gdf_osyu

##5840と5841の100メートルメッシュをconcatする

In [8]:
gdf_5840 = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/L03-b-16_5840-jgd_GML/L03-b-16_5840.shp', encoding='shift_jis')

In [9]:
gdf_5841 = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/L03-b-16_5841-jgd_GML/L03-b-16_5841.shp', encoding='shift_jis')

In [10]:
gdf_osyu_100m_mesh = pd.concat([gdf_5840, gdf_5841], ignore_index=True)

In [11]:
gdf_osyu_polygon = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/N03-20230101_03_GML/N03-23_03_230101.shp', encoding='shift-jis')

##重心座標を求める

In [12]:
gdf_osyu_100m_mesh = center(gdf_osyu_100m_mesh, gdf_osyu_polygon)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
gdf_osyu_100m_mesh.to_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp')

##天気

In [13]:
#天気気温のデータをダウンロード
gdf_weather_5840 = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/G02-22_5840-jgd_GML/G02-22_5840-jgd.shp')
gdf_weather_5841 = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/G02-22_5841-jgd_GML/G02-22_5841-jgd.shp')

In [14]:
#天気気温の5840と5841を空間結合
gdf_weather = pd.concat([gdf_weather_5840, gdf_weather_5841], ignore_index=True)

In [17]:
#100mメッシュに気温のデータを結合
gdf_osyu_100m_mesh = tempreture_rain(gdf_osyu_100m_mesh, gdf_weather)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
# 重複したインデックスを表示する
if gdf_osyu.index.duplicated().any():
    print("重複したインデックス:", gdf_osyu.index[gdf_osyu.index.duplicated()])
else:
    print("重複したインデックスはありません。")

重複したインデックスはありません。


In [18]:
gdf_osyu_100m_mesh.to_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/gdf_osyu_merge_weather.shp')

<ipython-input-18-6de6aca3aa46>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_osyu_100m_mesh.to_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/gdf_osyu_merge_weather.shp')


##道路

In [19]:
gdf_road = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/全国道路施設/drive-download-20230619T135611Z-001/道路舗装劣化データ_岩手県_utf8.csv', encoding='utf-8')

In [20]:
#道路点検データのカラム名を変更する。renameメソッドはrename()のかっこの中でcolumnsと指定しないとカラムは変更されない。
gdf_road = gdf_road.rename(columns = {'inspec_soundness':'paved_rank', 'inspec_soundness_name':'paved_rank_name'})

In [21]:
#道路劣化調査の奥州市のみのデータフレームを作る
gdf_road_osyu = gdf_road[gdf_road['locate_area2'] == '奥州市']
gdf_road_osyu = gpd.GeoDataFrame(gdf_road_osyu)

#データフレームの緯度経度座標をstr型からflaot型に変換
gdf_road_osyu['locate_self_long'] = gdf_road_osyu['locate_self_long'].replace({'':0}).astype(float)
gdf_road_osyu['locate_self_lat'] = gdf_road_osyu['locate_self_lat'].replace({'':0}).astype(float)

#データフレームの緯度経度座標が空の行を削除する
gdf_road_osyu = gdf_road_osyu[gdf_road_osyu['locate_self_lat'] != 0]

#緯度経度座標をpoint型にしたカラムを増やす
gdf_road_osyu['geometry'] = gdf_road_osyu.apply(lambda x: Point(x['locate_self_long'], x['locate_self_lat']), axis=1)

#座標系を設定
if gdf_road_osyu.crs is None:
    # 座標系をEPSG:4326に設定
    gdf_road_osyu.set_crs(epsg=4326, inplace=True)

In [22]:
#ポリゴンデータにポイントデータがあれば空間結合
gdf_osyu_100m_mesh = gpd.sjoin(gdf_osyu_100m_mesh, gdf_road_osyu, op='contains',  how='left')
#2つのデータフレームで同じカラム（geometry)がないと結合することが出来ない

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [23]:
#indexが重複しているものを削除する
gdf_osyu_100m_mesh = gdf_osyu_100m_mesh[~gdf_osyu_100m_mesh.index.duplicated(keep='first')]

#indexが重複しているものがないか確認
gdf_osyu_100m_mesh[gdf_osyu_100m_mesh.index.duplicated(keep=False)]

,Meshcode,lu_code,lu_year,geometry,longitude,...,paved_rank,paved_rank_name,sochi_state,sochi_state_name,facility_kind


In [24]:
#結合した際にfiltered_gdf_road_osyuのindexを削除　結合したときのindexを残したまま別のデータフレームは結合できない
gdf_osyu_100m_mesh = gdf_osyu_100m_mesh.drop('index_right', axis=1)

In [25]:
gdf_osyu_100m_mesh.to_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/全国道路施設/gdf_osyu_merge_road.shp')

<ipython-input-25-bd9c07fdfb76>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_osyu_100m_mesh.to_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/全国道路施設/gdf_osyu_merge_road.shp')


##騒音

In [26]:
#環境省自動車騒音データのダウンロード
car_noise = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/環境省自動車騒音データ/20230526215229/point.csv', encoding='shift-jis')

In [27]:
#カラムを変更する

#現在のカラムを変数に定義
car_noise_columns = car_noise.columns
#新しく設定するカラムを変数に定義
columns_name = ['Report Year',
                'surveyed_municipality',
                'serial_number',
                'Noise_measurement point',
                'Address_of_numbering_point',
                '[1]route_name',
                '[1]number_of_lanes',
                '[1]road_classification',
                '[2]route_name',
                '[2]number_of_lanes',
                '[2]road_classification',
                'Date_of_start_of_measurement',
                'Date_of_measurement_completion',
                'Roadway_edge_distance(m)',
                'Road_site_boundary_distance(m)',
                'Height_above_ground(m)',
                'environmental_standard type',
                'noise_rank_Daytime(dB)',
                'noise_rank_nighttime(dB)',
                'noise_latitude',
                'noise_longitude']
#二つのリストをzip関数でまとめforで一つづつ辞書に入れていく
car_noise = car_noise.rename(columns = {k: v for k, v in zip(car_noise.columns, columns_name)})

In [28]:
#座標系を設定
if car_noise.crs is None:
    # 座標系をEPSG:4326に設定
    car_noise.set_crs(epsg=4326, inplace=True)

#自動車騒音データを奥州市に絞る
car_noise_osyu = car_noise[car_noise['Address_of_numbering_point'].str.contains('奥州市', na=False)] #na引数をna=Falseにすることで欠損値を無視することができる

#geometryカラムにpointを付与
car_noise_osyu['geometry'] = car_noise_osyu.apply(lambda x: Point(x['noise_longitude'], x['noise_latitude']), axis=1) #緯度経度座標の順番が違うと結合することが出来ない。

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [29]:
#ポリゴンデータにポイントデータがあれば空間結合
gdf_osyu_100m_mesh = gpd.sjoin(gdf_osyu_100m_mesh, car_noise_osyu, op='contains', how='left')
#2つのデータフレームで同じカラム（geometry)がないと結合することが出来ない

#indexが重複しているものを削除する
gdf_osyu_100m_mesh = gdf_osyu_100m_mesh[~gdf_osyu_100m_mesh.index.duplicated(keep='first')]
#car_noise_osyuには6行重複があった。

#indexが重複しているものがないか確認
print(gdf_osyu_100m_mesh[gdf_osyu_100m_mesh.index.duplicated(keep=False)])#duplicated(keep=False)とすることで重複しているindexをTrueとしてその行を抜き出す

#結合した際にfiltered_gdf_road_osyuのindexを削除　結合したときのindexを残したまま別のデータフレームは結合できない
gdf_osyu_100m_mesh = gdf_osyu_100m_mesh.drop('index_right', axis=1)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


Empty GeoDataFrame
Columns: [Meshcode, lu_code, lu_year, geometry, longitude, latitude, G02_053, max_temperature, min_temperature, avg_rainfall, max_rainfall, facility_id, road_manager_c, road_manager_c_name, structures_kind, facility_name, route_name, road_manager_div, road_manager_div_name, road_manager_name, road_manager_office, locate_area1, locate_area1_name, locate_area2, locate_self_lat, locate_self_long, last_repair_year, length, pavement_type, inspec_year, paved_rank, paved_rank_name, sochi_state, sochi_state_name, facility_kind, index_right, Report Year, surveyed_municipality, serial_number, Noise_measurement point, Address_of_numbering_point, [1]route_name, [1]number_of_lanes, [1]road_classification, [2]route_name, [2]number_of_lanes, [2]road_classification, Date_of_start_of_measurement, Date_of_measurement_completion, Roadway_edge_distance(m), Road_site_boundary_distance(m), Height_above_ground(m), environmental_standard type, noise_rank_Daytime(dB), noise_rank_nighttime(dB

In [30]:
gdf_osyu_100m_mesh[~gdf_osyu_100m_mesh['noise_latitude'].isna()]

,Meshcode,lu_code,lu_year,geometry,longitude,...,environmental_standard type,noise_rank_Daytime(dB),noise_rank_nighttime(dB),noise_latitude,noise_longitude
37455,5841404887,0100,20161019,"POLYGON ((141.10875 39.04000, 141.10875 39.04083, 141.11000 39.04083, 141.11000 39.04000, 141.10875 39.04000))",141.109375,...,X,71,67,39.040516934880664,141.10958048235128
37482,5841404914,0100,20161019,"POLYGON ((141.11750 39.03417, 141.11750 39.03500, 141.11875 39.03500, 141.11875 39.03417, 141.11750 39.03417))",141.118125,...,X,77,73,39.03472426709791,141.11816517500134
38374,5841405806,0100,20161019,"POLYGON ((141.10750 39.04167, 141.10750 39.04250, 141.10875 39.04250, 141.10875 39.04167, 141.10750 39.04167))",141.108125,...,X,70,62,39.04200811769314,141.10775672440374
39556,5841406988,0700,20161019,"POLYGON ((141.12250 39.05667, 141.12250 39.05750, 141.12375 39.05750, 141.12375 39.05667, 141.12250 39.05667))",141.123125,...,B,65,57,39.057036,141.123666
40497,5841407929,0700,20161019,"POLYGON ((141.12375 39.06000, 141.12375 39.06083, 141.12500 39.06083, 141.12500 39.06000, 141.12375 39.06000))",141.124375,...,B,63,55,39.06063521944445,141.12476689444443
...,...,...,...,...,...,...,...,...,...,...,...
76245,5841612493,0700,20160811,"POLYGON ((141.17875 39.19083, 141.17875 39.19167, 141.18000 39.19167, 141.18000 39.19083, 141.17875 39.19083))",141.179375,...,C,65,57,39.19107568055556,141.17984795
77098,5841613400,0700,20160811,"POLYGON ((141.17500 39.19167, 141.17500 39.19250, 141.17625 39.19250, 141.17625 39.19167, 141.17500 39.19167))",141.175625,...,C,67,59,39.19167720921309,141.17557573994043
99765,5841724066,0100,20160611,"POLYGON ((141.25750 39.28833, 141.25750 39.28917, 141.25875 39.28917, 141.25875 39.28833, 141.25750 39.28833))",141.258125,...,X,68,65,39.288710469312775,141.2579655995329
99869,5841724170,0100,20160611,"POLYGON ((141.26250 39.28917, 141.26250 39.29000, 141.26375 39.29000, 141.26375 39.28917, 141.26250 39.28917))",141.263125,...,X,65,60,39.289808230101166,141.2635679042715


In [32]:
gdf_osyu_100m_mesh.to_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/環境省自動車騒音データ/gdf_osyu_merge_noise.shp')

<ipython-input-32-fd602e263934>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_osyu_100m_mesh.to_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/環境省自動車騒音データ/gdf_osyu_merge_noise.shp')


###騒音データが正確に結合されているかの確認

In [28]:
# ポイントデータを定義する
point = Point(141.10958048235128, 39.040516934880664)
# ポリゴンを定義する
polygon = Polygon([(141.10875, 39.04000), (141.10875, 39.04083), (141.11000, 39.04083), (141.11000, 39.04000), (141.10875, 39.04000)])

# ポイントがポリゴンの中に含まれているか判定する
is_within_polygon = polygon.contains(point)
print(is_within_polygon)

# 結果を表示する
if is_within_polygon:
    print("ポイントはポリゴンの中に含まれています。")
else:
    print("ポイントはポリゴンの中に含まれていません。")

True
ポイントはポリゴンの中に含まれています。


In [ ]:
gdf_osyu.shape

(101386, 56)

# 標高データの空間結合

In [29]:
import dask.dataframe as dd

df_elevation = dd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/DEM/03215.csv')

In [33]:
print(df_elevation.columns)

Index(['X', 'Y', 'fid', 'shubetsu', 'hyoukouchi', 'fid_2', 'KEY_CODE', 'PREF',
       'CITY', 'S_AREA', 'PREF_NAME', 'CITY_NAME', 'S_NAME', 'KIGO_E', 'HCODE',
       'AREA', 'PERIMETER', 'R2KAxx', 'R2KAxx_ID', 'KIHON1', 'DUMMY1',
       'KIHON2', 'KEYCODE1', 'KEYCODE2', 'AREA_MAX_F', 'KIGO_D', 'N_KEN',
       'N_CITY', 'KIGO_I', 'KBSUM', 'JINKO', 'SETAI', 'X_CODE', 'Y_CODE',
       'KCODE1'],
      dtype='object')


In [ ]:
df_elevation = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/DEM/03215.csv')

<ipython-input-29-a257be770230>:1: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_elevation = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/DEM/03215.csv')


In [ ]:
import subprocess
import dask.dataframe
import dask_geopandas
import download
from dask.distributed import Client, LocalCluster

ModuleNotFoundError: ignored

In [ ]:
df_elevation

,X,Y,fid,shubetsu,hyoukouchi,...,JINKO,SETAI,X_CODE,Y_CODE,KCODE1
0,141.129389,39.142833,71896638,その他,59.2,...,5748,2554,141.138372,39.148374,0010-01
1,141.129500,39.142833,71896639,その他,59.1,...,5748,2554,141.138372,39.148374,0010-01
2,141.129611,39.142833,71896640,その他,59.1,...,5748,2554,141.138372,39.148374,0010-01
3,141.129722,39.142833,71896641,その他,59.0,...,5748,2554,141.138372,39.148374,0010-01
4,141.129833,39.142833,71896642,その他,59.0,...,5748,2554,141.138372,39.148374,0010-01
...,...,...,...,...,...,...,...,...,...,...,...
8385351,141.126389,39.056056,80281989,その他,29.4,...,303,105,141.126391,39.057100,0810-04
8385352,141.126500,39.056056,80281990,その他,29.6,...,303,105,141.126391,39.057100,0810-04
8385353,141.126611,39.056056,80281991,その他,29.7,...,303,105,141.126391,39.057100,0810-04
8385354,141.126722,39.056056,80281992,その他,29.8,...,303,105,141.126391,39.057100,0810-04


In [ ]:
#gdf_elevationに座標系はついていないので付与
if gdf_elevation.crs is None:
	gdf_elevation.set_crs(epsg=4326, inplace=True)

NameError: ignored

In [ ]:
import dask.dataframe as dd

# Daskデータフレームを作成
ddf_osyu = dd.from_pandas(gdf_osyu, npartitions=10)

# GeoPandasデータフレームを作成
gdf_elevation = gpd.GeoDataFrame(df_elevation)

# カラムを絞る
ddf_elevation = gdf_elevation[['X', 'Y', 'hyoukouchi']]

# カラムの名称を変更
ddf_elevation = ddf_elevation.rename(columns={'X': 'longitude', 'Y': 'latitude'})

# geometryカラムを追加
ddf_elevation['geometry'] = dd.from_pandas(gpd.GeoSeries([Point(x, y) for x, y in zip(ddf_elevation['longitude'], ddf_elevation['latitude'])]), npartitions=10)

# 標高のデータを結合
gdf_osyu = gpd.sjoin(gdf_osyu.compute(), gdf_elevation.compute(), how='left', op='contains')

AttributeError: ignored

In [ ]:
#標高の値をグループ演算で平均、最大、最低を求める
grouped_df = df.groupby('group').agg({'A': 'mean', 'B': ['max', 'min']}).reset_index()

In [ ]:
# #出力するためのコード

# # エンコードできない文字を別の形式に変換してからShift_JISでエンコードする
# #gdf_5841を変えるのはめんどくさいからgdf_5841_outputに変数を格納する
# gdf_5841_output = gdf_5841
# gdf_5841_output['paved_rank_name'] = gdf_5841_output['paved_rank_name'].replace({'Ⅰ':'1', 'Ⅱ':2, 'Ⅲ':3 }, regex=True)  # 文字の置換
# # gdf_5841_output['column_name'] = gdf_5841_output['column_name'].str.encode('shift_jis', errors='ignore')  # Shift_JISでエンコード

# gdf_5841.to_csv('H:/マイドライブ/水道管劣化予測/result/merge_file.csv', index=False, encoding='shift-jis')